
# Convert **Tract Baseline → ZIP Baseline** (Population-Weighted)

**Upload two files when prompted:**
1. `baseline_snapshot_Boston_tract.csv`  
2. A **Tract→ZIP crosswalk** CSV (HUD crosswalk recommended).

**Output:** `baseline_snapshot_Boston_ZIP.csv`


In [ ]:

# ⬆️ Upload in Colab
import pandas as pd, numpy as np

try:
    from google.colab import files
    print("Upload baseline_snapshot_Boston_tract.csv")
    up1 = files.upload()
    tract_path = list(up1.keys())[0]
    print("Upload tract→ZIP crosswalk CSV")
    up2 = files.upload()
    xwalk_path = list(up2.keys())[0]
except Exception as e:
    tract_path = "baseline_snapshot_Boston_tract.csv"
    xwalk_path = "TRACT_ZIP_CROSSWALK.csv"

tract = pd.read_csv(tract_path)
xw = pd.read_csv(xwalk_path)

# normalize keys
def only_digits(s):
    return ''.join([ch for ch in str(s) if ch.isdigit()])

xw_cols = [c for c in xw.columns]
# pick tract id column
cand = [c for c in xw_cols if c.lower() in ["geoid","tractfips","tractce","tract","tract id","tract_id"]]
if not cand: raise ValueError("Crosswalk missing tract id column (e.g., GEOID/TRACTFIPS).")
xw_key = cand[0]
xw[xw_key] = xw[xw_key].map(only_digits)

tract_key = "GEOID" if "GEOID" in tract.columns else None
if tract_key is None:
    cand2 = [c for c in tract.columns if "TRACT" in c.upper() or "GEO" in c.upper()]
    if not cand2: raise ValueError("Tract baseline needs GEOID.")
    tract_key = cand2[0]
tract[tract_key] = tract[tract_key].map(only_digits)

# choose weight column
w_cand = [c for c in xw.columns if ("RES" in c.upper()) and ("RATIO" in c.upper() or "WEIGHT" in c.upper())]
weight_col = w_cand[0] if w_cand else None

m = tract.merge(xw, left_on=tract_key, right_on=xw_key, how="inner")

if weight_col is None:
    if "Population" not in m.columns: raise ValueError("Need a weight column or Population in baseline.")
    m["_w"] = m["Population"]
else:
    m["_w"] = pd.to_numeric(m[weight_col], errors="coerce").fillna(0.0)
    if "Population" in m.columns:
        m["_w"] = m["_w"] * m["Population"]

# numeric columns to weight-average
exclude = {tract_key, xw_key, "ZIP", weight_col if weight_col else ""}
num_cols = [c for c in tract.columns if c not in exclude and pd.api.types.is_numeric_dtype(tract[c])]

def agg_zip(g):
    out = {}
    W = g["_w"].sum()
    for c in num_cols:
        out[c] = (g[c]*g["_w"]).sum()/W if W>0 else g[c].mean()
    if "Population" in g.columns:
        out["Population"] = g["Population"].sum()
    return pd.Series(out)

zip_baseline = m.groupby("ZIP").apply(agg_zip).reset_index()

zip_baseline.to_csv("baseline_snapshot_Boston_ZIP.csv", index=False)
print("✅ Wrote baseline_snapshot_Boston_ZIP.csv with", len(zip_baseline), "rows")
